## Install dependencies

In [ ]:
!pip install -U sagemaker==2.192.1

## Build Custom Container
First we augment the Dockerfile for SageMaker.

In [ ]:
sm_entry_stmt = """
# Text Generation Inference base env
ENV HUGGINGFACE_HUB_CACHE=/tmp \
    HF_HUB_ENABLE_HF_TRANSFER=1 \
    PORT=80
COPY sagemaker-entrypoint.sh entrypoint.sh
RUN chmod +x entrypoint.sh

ENTRYPOINT ["./entrypoint.sh"]
CMD [ "" ]
"""

In [ ]:
with open("../tgi-custom/Dockerfile", "r") as fin:
    docker_content = fin.read()

sm_docker_cotent = docker_content + sm_entry_stmt

with open("Dockerfile_sm", "w") as fout:
    fout.write(sm_docker_cotent)

Then we build the image. This could take 10 minutes, so feel free to run it directly in the terminal in case the notebook cell times out.  

**Important Note** - Please ensure the `ROLE` has sufficient permission to push Docker images to Elastic Container Registry.

In [ ]:
!cp -r ../tgi-custom/vllm ./vllm

In [ ]:
REPO_NAME = "mistrallite-tgi110-ecr"

In [ ]:
!bash sm_build.sh {REPO_NAME}

## Deploy SageMaker Endpoint

In [ ]:
import boto3
import json
import sagemaker
import time

from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

def get_aws_region():
    # Get the current AWS region from the default session
    session = boto3.session.Session()
    return session.region_name

def get_aws_account_id():
    # Get the current AWS account ID from the default session
    sts_client = boto3.client("sts")
    response = sts_client.get_caller_identity()
    return response["Account"]

REGION = get_aws_region()
ACCOUNT_ID = get_aws_account_id()

role = sagemaker.get_execution_role()

In [ ]:
image_uri = f"{ACCOUNT_ID}.dkr.ecr.{REGION}.amazonaws.com/{REPO_NAME}"
image_uri

In [ ]:
model_name = "MistralLite-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

instance_type = "ml.g5.2xlarge"
num_gpu = 1
max_input_length = 24000
max_total_tokens = 24576

hub = {
    'HF_MODEL_ID':'amazon/MistralLite',
    'HF_TASK':'text-generation',
    'SM_NUM_GPUS': json.dumps(num_gpu),
    "MAX_INPUT_LENGTH": json.dumps(max_input_length),
    "MAX_TOTAL_TOKENS": json.dumps(max_total_tokens),
    "MAX_BATCH_PREFILL_TOKENS": json.dumps(max_total_tokens),
    "MAX_BATCH_TOTAL_TOKENS":  json.dumps(max_total_tokens),
    "DTYPE": 'bfloat16',
}

model = HuggingFaceModel(
    name=model_name,
    env=hub,
    role=role,
    image_uri=image_uri
)
predictor = model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  endpoint_name=model_name,   
)

## Perform Inference

### Sagemaker SDK

In [ ]:
input_data = {
  "inputs": "<|prompter|>What are the main challenges to support a long context for LLM?</s><|assistant|>",
  "parameters": {
    "do_sample": False,
    "max_new_tokens": 400,
    "return_full_text": False,
    #"typical_p": 0.2,
    #"temperature":None,
    #"truncate":None,
    #"seed": 1,
  }
}
result = predictor.predict(input_data)[0]["generated_text"]
print(result)

### boto3

In [ ]:
import boto3
import json
def call_endpoint(client, prompt, endpoint_name, paramters):
    client = boto3.client("sagemaker-runtime")
    payload = {"inputs": prompt,
               "parameters": parameters}
    response = client.invoke_endpoint(EndpointName=endpoint_name,
                                      Body=json.dumps(payload), 
                                      ContentType="application/json")
    output = json.loads(response["Body"].read().decode())
    result = output[0]["generated_text"]
    return result

client = boto3.client("sagemaker-runtime")
parameters = {
    "do_sample": False,
    "max_new_tokens": 400,
    "return_full_text": False,
    #"typical_p": 0.2,
    #"temperature":None,
    #"truncate":None,
    #"seed": 10,
}
endpoint_name = predictor.endpoint_name
prompt = "<|prompter|>What are the main challenges to support a long context for LLM?</s><|assistant|>"
result = call_endpoint(client, prompt, endpoint_name, parameters)
print(result)

Try the long context of over 13,400 tokens, which are copied from [Amazon Aurora FAQs](https://aws.amazon.com/rds/aurora/faqs/)

In [ ]:
with open("../example_long_ctx.txt", "r") as fin:
    task_instruction = fin.read()
    task_instruction = task_instruction.format(
        my_question="please tell me how does pgvector help with Generative AI and give me some examples."
    )
prompt = f"<|prompter|>{task_instruction}</s><|assistant|>"
result = call_endpoint(client, prompt, endpoint_name, parameters)
print(result)